In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

In [2]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200623


import datetime 
base = datetime.date(2020,2,1)
for i in range(0,10): 
    d=base + datetime.timedelta(i)
    d=d.strftime("%Y%m%d")
    print(f"N029kakutokuruiseki{d}.csv")

from datetime import datetime, timedelta

def date_range(start_date: datetime, end_date: datetime):
    diff = (end_date - start_date).days + 1
    return (start_date + timedelta(i) for i in range(diff))

In [3]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [4]:
#ディレクトリ
data_dir = Path(r"C:\Users\crie072\Documents\21_Online")


#承認月
approval_month = "6月"

#Input_file
#①承認リスト（当月分）
input_file_1 = "N029（2020年6月入会：6月19日承認分まで）.xlsx"

#②承認リスト（前月分）
input_file_2 = "N029（2020年5月度入会）.xlsx"

#③ デジガレ確認用リスト
input_file_3 = "20200621〆【JFRカード御中】成果データ&問い合わせ.xlsx"

#④ 累積承認リスト
input_file_4 = "アフィリエイト広告_累積承認者リスト.xlsx"

#Output_file
#①オンライン承認リスト
output_file_1 = f"online_{today_yyyymmdd}" 
print("Output①⇒",output_file_1)

#②アフィリエイト月次成果承認確定リスト
output_file_2 = f"確認済み成果データ_{today_yyyymmdd}" 
print("Output②⇒",output_file_2)

#③アフィリエイト広告_承認者リスト_累積
output_file_3 = f"アフィリエイト広告_承認者リスト_{today_yyyymmdd}" 
print("Output③⇒",output_file_3)

Output①⇒ online_20200623
Output②⇒ 確認済み成果データ_20200623
Output③⇒ アフィリエイト広告_承認者リスト_20200623


①承認リスト（当月分）

In [5]:
print(input_file_1)

df_app = pd.read_excel(data_dir /r"input"/f"{input_file_1}")
df_app.head()

N029（2020年6月入会：6月19日承認分まで）.xlsx


,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD
0,20200601,1,0,D00,8,31,1,0,8,170,1,0,NaN,NaN,NaN,NaN,155193007642727,29,12.0,10000,20.0,1.000052e+10,2,0
1,20200601,1,0,D02,1,2,1,0,3,102,1,0,NaN,NaN,NaN,NaN,155603602336855,58,12.0,2000,NaN,2.000010e+09,2,0
2,20200601,1,0,D02,4,2,2,0,3,102,1,0,NaN,NaN,NaN,NaN,155603602336459,40,12.0,2000,NaN,2.000010e+09,2,0
3,20200601,1,0,D02,1,2,1,0,3,102,1,0,NaN,NaN,NaN,NaN,155603602336426,49,12.0,2000,NaN,2.000010e+09,2,0
4,20200601,D,0,D00,50,31,0,0,8,103,2,0,NaN,NaN,NaN,NaN,155203047717999,72,12.0,3000,20.0,3.000000e+03,1,0


In [6]:
print(df_app.shape)
print(df_app.index)
print(df_app.columns)
print(df_app.dtypes)

(3361, 24)
RangeIndex(start=0, stop=3361, step=1)
Index(['カード加入日', '会員カード発行区分', '続柄', 'カード提携会社1CD', 'カード提携会社2CD', 'カード種類CD',
       'カードブランドCD', '切替CD', '勧誘CD', '受付部支店', '性別CD', '再発行CD', '旧カード提携会社1CD',
       '旧カード提携会社2CD', '旧カード種類CD', '旧カードブランドCD', '入会申込番号', '満年齢', 'リボ支払方法CD',
       '勧誘団体CD', '提携カード特殊エリア', 'エンボス内容', 'カード年会費CD', 'リボ利用対象CD'],
      dtype='object')
カード加入日           int64
会員カード発行区分       object
続柄               int64
カード提携会社1CD      object
カード提携会社2CD       int64
カード種類CD          int64
カードブランドCD        int64
切替CD             int64
勧誘CD             int64
受付部支店            int64
性別CD             int64
再発行CD            int64
旧カード提携会社1CD     object
旧カード提携会社2CD    float64
旧カード種類CD       float64
旧カードブランドCD     float64
入会申込番号           int64
満年齢              int64
リボ支払方法CD       float64
勧誘団体CD           int64
提携カード特殊エリア     float64
エンボス内容         float64
カード年会費CD         int64
リボ利用対象CD         int64
dtype: object


column_names = ['実績月','入会申込番号','直近３ケ月買物利用金額','直近３ケ月全体利用金額','年令','性別',
                '都道府県名','顧客状況コ－ド','マイペ設定','DM希望店','カード券種名','カ－ド加入日','初回利用日',
                'あとからリボ申込有無コード','Weｂ明細有無コード','解約','入会チャネル']

オンライン入会客を特定

In [7]:
df_app =df_app[(df_app["会員カード発行区分"] == "1") 
               & (df_app["続柄"] == 0) 
               & ~(df_app["カード提携会社2CD"] == 95) 
               & (df_app["切替CD"] == 0) 
               & (df_app["受付部支店"] == 299)]

In [8]:
#含まれてないか確認
print(df_app["会員カード発行区分"].unique())
print(df_app["続柄"].unique())
print(df_app["カード提携会社2CD"].unique())
print(df_app["切替CD"].unique())
print(df_app["受付部支店"].unique())
df_app.shape[0]

['1']
[0]
[ 1  8  4 11]
[0]
[299]


484

In [9]:
#"成果"カラムを追加
df_app['成果'] = "可"
df_app['参照用'] = df_app['入会申込番号'] #"参照用"はデジガレとのマッチング用

In [10]:
df_app=df_app[['入会申込番号','成果','カード加入日','参照用']]
df_app.head()

,入会申込番号,成果,カード加入日,参照用
16,155051921901571,可,20200601,155051921901571
21,155051921904138,可,20200601,155051921904138
28,155051921870925,可,20200601,155051921870925
43,155051921904971,可,20200601,155051921904971
46,155051921880270,可,20200601,155051921880270


In [11]:
#csvアウトプット
df_tmp = df_app.drop(columns=['参照用'])
df_tmp.to_csv(data_dir /r"output"/f"{output_file_1}.csv", index=False,header=True,encoding='utf-8-sig')

②前月の承認結果作業も読み込む

In [12]:
print(input_file_2)

df_pre = pd.read_excel(data_dir /r"input"/f"{input_file_2}")
df_pre.head()

N029（2020年5月度入会）.xlsx


,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD
0,20200501,1,0,D02,1,2,1,0,3,102,1,0,NaN,NaN,NaN,NaN,155603602331567,48,12.0,2000,NaN,2.000010e+09,2,0
1,20200501,1,0,D00,95,0,0,0,3,153,2,0,NaN,NaN,NaN,NaN,155603602331427,61,NaN,0,NaN,NaN,1,0
2,20200501,1,0,D00,95,0,0,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921869703,54,NaN,0,NaN,NaN,1,0
3,20200501,1,0,D00,1,31,1,0,8,170,2,0,NaN,NaN,NaN,NaN,155193007662402,36,12.0,10000,20.0,1.000071e+10,2,0
4,20200501,1,0,D00,1,31,1,0,8,153,2,0,NaN,NaN,NaN,NaN,155193027692611,74,12.0,53000,20.0,5.300000e+04,2,0


オンライン入会客を特定

In [13]:
df_pre =df_pre[(df_pre["会員カード発行区分"] == "1") 
               & (df_pre["続柄"] == 0) 
               & ~(df_pre["カード提携会社2CD"] == 95) 
               & (df_pre["切替CD"] == 0) 
               & (df_pre["受付部支店"] == 299)]

In [14]:
#含まれてないか確認
print(df_pre["会員カード発行区分"].unique())
print(df_pre["続柄"].unique())
print(df_pre["カード提携会社2CD"].unique())
print(df_pre["切替CD"].unique())
print(df_pre["受付部支店"].unique())
df_pre.shape[0]

['1']
[0]
[ 1  8  4 18 14 11]
[0]
[299]


585

In [15]:
#"成果"カラムを追加
df_pre['成果'] = "可"
df_pre['参照用'] = df_pre['入会申込番号'] #"参照用"はデジガレとのマッチング用

In [16]:
df_pre=df_pre[['入会申込番号','成果','カード加入日','参照用']]
df_pre.head()

,入会申込番号,成果,カード加入日,参照用
5,155051921812315,可,20200501,155051921812315
16,155051921869588,可,20200501,155051921869588
20,155051921870073,可,20200501,155051921870073
31,155051921791964,可,20200501,155051921791964
35,155051921869471,可,20200501,155051921869471


In [17]:
#前月と当月の承認リストをドッキング
df_app = pd.concat([df_pre, df_app])
df_app.shape[0]

1069

In [18]:
print(df_app['カード加入日'].unique())

[20200501 20200507 20200508 20200511 20200512 20200513 20200514 20200515
 20200518 20200519 20200520 20200521 20200522 20200525 20200526 20200527
 20200528 20200529 20200601 20200602 20200603 20200604 20200605 20200608
 20200609 20200610 20200611 20200612 20200615 20200616 20200617 20200618
 20200619]


③ デジガレ確認用リスト

In [86]:
print(input_file_3)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file_3}", sheet_name = 0)

#シート名取得
bk = pd.ExcelFile(data_dir/r"input"/f"{input_file_3}")
print(bk.sheet_names)

df_dg.head()

20200621〆【JFRカード御中】成果データ&問い合わせ.xlsx
['成果データ', '識別子不明', '成果データ（5月～利用条件追加媒体）']


,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,成果,カード発行日
0,1776418,Tモール,ポイント,2020-02-23 07:09:00,2020-02-23 07:26:00,155051921747735,JA,9000,NaN,NaN
1,619,モッピー（連携用）,ポイント,2020-02-23 19:14:23,2020-02-23 19:26:04,155051921748824,BGT,9000,NaN,NaN
2,619,モッピー（連携用）,ポイント,2020-02-24 07:49:13,2020-02-24 07:56:03,155051921749541,BGT,9000,NaN,NaN
3,49,ハピタス（連携用）,ポイント,2020-02-25 06:24:33,2020-02-25 06:38:04,155051921751422,BGT,9000,NaN,NaN
4,619,モッピー（連携用）,ポイント,2020-02-25 10:55:00,2020-02-25 11:02:04,155051921751596,BGT,9000,NaN,NaN


In [88]:
print(bk.sheet_names[0])

成果データ


シート1の対応

In [53]:
#余計なカラム削除
#df_dg = df_dg.drop(columns=['Unnamed: 9','成果'])
df_dg = df_dg.drop(columns=['成果'])

In [54]:
#ASIDの整数化
for i in range(len(df_dg)):
    if pd.isnull(df_dg['ASID'][i]) == True:
        df_dg.loc[:,'ASID'][i] = 0
df_dg = df_dg.astype({'ASID':int})

In [55]:
print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(839, 9)
RangeIndex(start=0, stop=839, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '参照用', 'ASP名', '単価', 'カード発行日'], dtype='object')
ASID               int32
メディア名             object
媒体チャネル            object
クリック              object
申込        datetime64[ns]
参照用                int64
ASP名              object
単価                 int64
カード発行日           float64
dtype: object


In [56]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード発行日,申込年月
0,1776418,Tモール,ポイント,2020-02-23 07:09:00,2020-02-23 07:26:00,155051921747735,JA,9000,NaN,2020年2月
1,619,モッピー（連携用）,ポイント,2020-02-23 19:14:23,2020-02-23 19:26:04,155051921748824,BGT,9000,NaN,2020年2月
2,619,モッピー（連携用）,ポイント,2020-02-24 07:49:13,2020-02-24 07:56:03,155051921749541,BGT,9000,NaN,2020年2月
3,49,ハピタス（連携用）,ポイント,2020-02-25 06:24:33,2020-02-25 06:38:04,155051921751422,BGT,9000,NaN,2020年2月
4,619,モッピー（連携用）,ポイント,2020-02-25 10:55:00,2020-02-25 11:02:04,155051921751596,BGT,9000,NaN,2020年2月


In [57]:
#承認データ（df_app）をマージする前に、df_app余計なカラムを削除
df_app_tmp = df_app.drop(columns=['入会申込番号'])

#承認データの"成果"をマージ
df_dg2 = pd.merge(df_dg,df_app_tmp, on =['参照用'], how = 'left')
df_dg2[df_dg2['成果'] == '可'].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード発行日,申込年月,成果,カード加入日
9,128,Warau.jp,ポイント,2020-03-13 17:32:49,2020-03-13 17:42:05,155051921783417,BGT,5000,NaN,2020年3月,可,20200602.0
15,49,ハピタス（連携用）,ポイント,2020-03-19 07:55:33,2020-03-19 08:14:04,155051921794224,BGT,5000,NaN,2020年3月,可,20200602.0
16,619,モッピー（連携用）,ポイント,2020-03-20 12:00:24,2020-03-20 12:25:04,155051921796021,BGT,5000,NaN,2020年3月,可,20200617.0
17,32190,ポイぷる,ポイント,2020-03-21 11:33:49,2020-03-21 11:45:05,155051921797300,ADV,5000,NaN,2020年3月,可,20200602.0
25,57,ライフメディア,ポイント,2020-03-25 00:36:26,2020-03-25 00:48:04,155051921806424,BGT,5000,NaN,2020年3月,可,20200526.0


In [58]:
df_dg2 = df_dg2.copy()

In [60]:
#マージしたらカード加入日がfloat型になってしまったため、整数型に戻す
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['カード加入日'][i]) == True:
        df_dg2.loc[:,'カード加入日'][i] = 0
        
df_dg2 = df_dg2.astype({'カード加入日':int})

In [61]:
df_dg2[df_dg2['カード加入日'] == 0].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード発行日,申込年月,成果,カード加入日
0,1776418,Tモール,ポイント,2020-02-23 07:09:00,2020-02-23 07:26:00,155051921747735,JA,9000,NaN,2020年2月,NaN,0
1,619,モッピー（連携用）,ポイント,2020-02-23 19:14:23,2020-02-23 19:26:04,155051921748824,BGT,9000,NaN,2020年2月,NaN,0
2,619,モッピー（連携用）,ポイント,2020-02-24 07:49:13,2020-02-24 07:56:03,155051921749541,BGT,9000,NaN,2020年2月,NaN,0
3,49,ハピタス（連携用）,ポイント,2020-02-25 06:24:33,2020-02-25 06:38:04,155051921751422,BGT,9000,NaN,2020年2月,NaN,0
4,619,モッピー（連携用）,ポイント,2020-02-25 10:55:00,2020-02-25 11:02:04,155051921751596,BGT,9000,NaN,2020年2月,NaN,0


In [63]:
#"成果"がNanのものは[保留]に置換する
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['成果'][i]) == True:
        df_dg2.loc[:,'成果'][i] = "保留"

In [64]:
df_dg2.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード発行日,申込年月,成果,カード加入日
0,1776418,Tモール,ポイント,2020-02-23 07:09:00,2020-02-23 07:26:00,155051921747735,JA,9000,NaN,2020年2月,保留,0
1,619,モッピー（連携用）,ポイント,2020-02-23 19:14:23,2020-02-23 19:26:04,155051921748824,BGT,9000,NaN,2020年2月,保留,0
2,619,モッピー（連携用）,ポイント,2020-02-24 07:49:13,2020-02-24 07:56:03,155051921749541,BGT,9000,NaN,2020年2月,保留,0
3,49,ハピタス（連携用）,ポイント,2020-02-25 06:24:33,2020-02-25 06:38:04,155051921751422,BGT,9000,NaN,2020年2月,保留,0
4,619,モッピー（連携用）,ポイント,2020-02-25 10:55:00,2020-02-25 11:02:04,155051921751596,BGT,9000,NaN,2020年2月,保留,0


集計

In [65]:
#客数
df_dg2[df_dg2['成果'] == '可'].shape[0]

318

In [66]:
pd.crosstab(df_dg2['申込年月'],df_dg2['成果'])

成果,保留,可
申込年月,,
2020年1月,3,0
2020年2月,8,0
2020年3月,29,6
2020年4月,87,6
2020年5月,188,215
2020年6月,206,91


In [100]:
#CSVアウトプット
df_dg2.to_csv(data_dir /r"output"/f"{output_file_2}_{bk.sheet_names[0]}.csv", index=False,header=True,encoding='utf-8-sig')

In [89]:
print(bk.sheet_names[2])

成果データ（5月～利用条件追加媒体）


③ デジガレ確認用リスト ※別シート

In [67]:
print(input_file_3)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file_3}", sheet_name = 2)
df_dg.head()

20200621〆【JFRカード御中】成果データ&問い合わせ.xlsx


,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,成果,カード発行日
0,37,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,NaN,NaN
1,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,NaN,NaN
2,37,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,NaN,NaN
3,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,NaN,NaN
4,37,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000,NaN,NaN


In [68]:
#余計なカラム削除
#df_dg = df_dg.drop(columns=['Unnamed: 9','成果'])
df_dg = df_dg.drop(columns=['成果'])

In [69]:
#ASIDの整数化
for i in range(len(df_dg)):
    if pd.isnull(df_dg['ASID'][i]) == True:
        df_dg.loc[:,'ASID'][i] = 0
df_dg = df_dg.astype({'ASID':int})

In [70]:
print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(555, 9)
RangeIndex(start=0, stop=555, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '参照用', 'ASP名', '単価', 'カード発行日'], dtype='object')
ASID               int32
メディア名             object
媒体チャネル            object
クリック      datetime64[ns]
申込        datetime64[ns]
参照用                int64
ASP名              object
単価                 int64
カード発行日           float64
dtype: object


In [71]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード発行日,申込年月
0,37,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,NaN,2020年5月
1,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,NaN,2020年5月
2,37,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,NaN,2020年5月
3,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,NaN,2020年5月
4,37,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000,NaN,2020年5月


In [72]:
#承認データ（df_app）をマージする前に、df_app余計なカラムを削除
df_app_tmp = df_app.drop(columns=['入会申込番号'])

#承認データの"成果"をマージ
df_dg2 = pd.merge(df_dg,df_app_tmp, on =['参照用'], how = 'left')
df_dg2[df_dg2['成果'] == '可'].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード発行日,申込年月,成果,カード加入日
1,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,NaN,2020年5月,可,20200615.0
3,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,NaN,2020年5月,可,20200611.0
5,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,NaN,2020年5月,可,20200525.0
6,21,ポイントインカム,ポイント,2020-05-01 07:24:26,2020-05-01 07:35:04,155051921869471,BGT,9000,20200501.0,2020年5月,可,20200501.0
8,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,NaN,2020年5月,可,20200525.0


In [73]:
df_dg2 = df_dg2.copy()

In [75]:
#マージしたらカード加入日がfloat型になってしまったため、整数型に戻す
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['カード加入日'][i]) == True:
        df_dg2.loc[:,'カード加入日'][i] = 0
        
df_dg2 = df_dg2.astype({'カード加入日':int})

In [76]:
df_dg2[df_dg2['カード加入日'] == 0].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード発行日,申込年月,成果,カード加入日
0,37,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,NaN,2020年5月,NaN,0
2,37,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,NaN,2020年5月,NaN,0
4,37,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000,NaN,2020年5月,NaN,0
7,37,げん玉,ポイント,2020-05-01 07:46:58,2020-05-01 08:02:04,155051921869489,BGT,9000,NaN,2020年5月,NaN,0
14,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-05-01 11:33:44,2020-05-01 11:46:06,155051921869679,BGT,9000,NaN,2020年5月,NaN,0


In [78]:
#"成果"がNanのものは[保留]に置換する
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['成果'][i]) == True:
        df_dg2.loc[:,'成果'][i] = "保留"

In [79]:
df_dg2.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード発行日,申込年月,成果,カード加入日
0,37,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,NaN,2020年5月,保留,0
1,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,NaN,2020年5月,可,20200615
2,37,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,NaN,2020年5月,保留,0
3,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,NaN,2020年5月,可,20200611
4,37,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000,NaN,2020年5月,保留,0


集計

In [80]:
#客数
df_dg2[df_dg2['成果'] == '可'].shape[0]

364

In [81]:
pd.crosstab(df_dg2['申込年月'],df_dg2['成果'])

成果,保留,可
申込年月,,
2020年5月,90,310
2020年6月,101,54


In [99]:
#CSVアウトプット
df_dg2.to_csv(data_dir /r"output"/f"{output_file_2}_{bk.sheet_names[2]}.csv", index=False,header=True,encoding='utf-8-sig')

③累積承認リストを作成する

In [37]:
#↑デジガレ報告用のファイルから余計なカラムを削除する
df_dg2_tmp = df_dg2.drop(columns=['申込年月','カード加入日'])
df_dg2_tmp['承認作業月'] = approval_month
df_dg2_tmp.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード発行日,成果,承認作業月
0,1776418,Tモール,ポイント,2020-02-23 07:09:00,2020-02-23 07:26:00,155051921747735,JA,9000,NaN,保留,6月
1,619,モッピー（連携用）,ポイント,2020-02-23 19:14:23,2020-02-23 19:26:04,155051921748824,BGT,9000,NaN,保留,6月
2,619,モッピー（連携用）,ポイント,2020-02-24 07:49:13,2020-02-24 07:56:03,155051921749541,BGT,9000,NaN,保留,6月
3,49,ハピタス（連携用）,ポイント,2020-02-25 06:24:33,2020-02-25 06:38:04,155051921751422,BGT,9000,NaN,保留,6月
4,619,モッピー（連携用）,ポイント,2020-02-25 10:55:00,2020-02-25 11:02:04,155051921751596,BGT,9000,NaN,保留,6月


In [38]:
print(input_file_4)

df_cum = pd.read_excel(data_dir/"input"/f"{input_file_4}", dtype={'参照用': str})
df_cum.head()

アフィリエイト広告_累積承認者リスト.xlsx


,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,可否,承認作業月
0,37.0,げん玉,ポイント,2020-01-08 19:11:08,2020-01-08 19:32:05,155051921643397,BGT,9000,可,1月
1,52.0,ECナビSP,ポイント,2020-01-08 20:21:44,2020-01-08 20:31:05,155051921643553,BGT,9000,可,1月
2,52.0,ECナビSP,ポイント,2020-01-08 20:31:50,2020-01-08 20:39:06,155051921643587,BGT,9000,可,1月
3,52.0,ECナビSP,ポイント,2020-01-08 20:34:03,2020-01-08 20:45:05,155051921643678,BGT,9000,可,1月
4,82.0,ちょびリッチ。ドットコム,ポイント,2020-01-08 20:32:12,2020-01-08 20:55:06,155051921643744,BGT,9000,可,1月


In [40]:
for i in range(len(df_cum)):
    if pd.isnull(df_cum['ASID'][i]) == True:
        df_cum.loc[:,'ASID'][i] = 0

df_cum = df_cum.astype({'ASID':int})

In [41]:
print(df_cum['ASID'].unique())

[     37      52      82      54     619    1375      21    5953     165
    5954     204      46      10       8      45      49      57     101
     164     183 1523381     166 1776418     128     280     284    8163
      18      44     282 1595028 1580002 1496955 1594905    5618   32190
    5955       0 1454503 1591985    6711]


In [42]:
df_cum.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,可否,承認作業月
0,37,げん玉,ポイント,2020-01-08 19:11:08,2020-01-08 19:32:05,155051921643397,BGT,9000,可,1月
1,52,ECナビSP,ポイント,2020-01-08 20:21:44,2020-01-08 20:31:05,155051921643553,BGT,9000,可,1月
2,52,ECナビSP,ポイント,2020-01-08 20:31:50,2020-01-08 20:39:06,155051921643587,BGT,9000,可,1月
3,52,ECナビSP,ポイント,2020-01-08 20:34:03,2020-01-08 20:45:05,155051921643678,BGT,9000,可,1月
4,82,ちょびリッチ。ドットコム,ポイント,2020-01-08 20:32:12,2020-01-08 20:55:06,155051921643744,BGT,9000,可,1月


In [43]:
df_dg2_tmp = df_dg2_tmp.rename(columns={'成果':'可否'})
df_dg2_tmp.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード発行日,可否,承認作業月
0,1776418,Tモール,ポイント,2020-02-23 07:09:00,2020-02-23 07:26:00,155051921747735,JA,9000,NaN,保留,6月
1,619,モッピー（連携用）,ポイント,2020-02-23 19:14:23,2020-02-23 19:26:04,155051921748824,BGT,9000,NaN,保留,6月
2,619,モッピー（連携用）,ポイント,2020-02-24 07:49:13,2020-02-24 07:56:03,155051921749541,BGT,9000,NaN,保留,6月
3,49,ハピタス（連携用）,ポイント,2020-02-25 06:24:33,2020-02-25 06:38:04,155051921751422,BGT,9000,NaN,保留,6月
4,619,モッピー（連携用）,ポイント,2020-02-25 10:55:00,2020-02-25 11:02:04,155051921751596,BGT,9000,NaN,保留,6月


In [44]:
df_concat = pd.concat([df_cum, df_dg2_tmp])

In [45]:
df_concat = df_concat[df_concat['可否'] == "可"]

In [46]:
df_concat.groupby(['承認作業月'])['参照用'].count()

承認作業月
1月    363
2月    788
3月    284
4月     79
5月    107
6月    318
Name: 参照用, dtype: int64

In [93]:
#CSVアウトプット
df_concat.to_csv(data_dir /"output"/f"{output_file_3}.csv", index=False,header=True,encoding='utf-8-sig')